In [1]:
 ##  TOYMAZE  ##

""" 
Laberinto juguete implementado como una matriz cuadrada de 0s y 1s.
Consideramos:
        0 para una casilla vacia
        1 para una casilla ocupada
"""

def createboard(row,col):
  X = []
  for i in range(row):
    if i == 0 or i == row-1:     
      X.append([1]*col)
    else:
      X.append([0]*col)
  for i in range(row):
    X[i][0] = 1
    X[i][col-1] = 1
  return X

def toymaze():
  row = 11
  col = 11
  maze = createboard(row,col)
  
  for i in range(2,7):
    maze[i][2] = 1
    maze[i][8] = 1
    maze[6][i] = 1

  for i in range(2,5):
    maze[i][4] = 1
    maze[i][6] = 1

  for i in range(6,10):
    maze[i][4] = 1

  for i in range(2,9):
    maze[8][i] = 1
  
  maze[8][5] = 0
  maze[5][0] = 0

  maze[2][5] = maze[4][3] = maze[4][7] =  1
  maze[9][4] = maze[9][8] = 1
  
  printmaze(maze)
  
  return maze

def printmaze(maze):
  
  print("\n")  
  for row in maze:
    draw = ""
    for cell in row:
      if cell:
        draw = draw+"++"
      else:
        draw = draw+"  "

    print(draw)
    print(draw)
  print("\n")
  return

toymaze()



++++++++++++++++++++++
++++++++++++++++++++++
++                  ++
++                  ++
++  ++  ++++++  ++  ++
++  ++  ++++++  ++  ++
++  ++  ++  ++  ++  ++
++  ++  ++  ++  ++  ++
++  ++++++  ++++++  ++
++  ++++++  ++++++  ++
    ++          ++  ++
    ++          ++  ++
++  ++++++++++  ++  ++
++  ++++++++++  ++  ++
++      ++          ++
++      ++          ++
++  ++++++  ++++++  ++
++  ++++++  ++++++  ++
++      ++      ++  ++
++      ++      ++  ++
++++++++++++++++++++++
++++++++++++++++++++++




[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [2]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [3]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}, parent_action: {}, depth {}>".format(self.state, self.action, self.depth)

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

In [4]:
class State( ):
  
  ori = ('N','S','E','W')
  
  def __init__(self,x,y,o):
    self.x = x
    self.y = y
    if o in self.ori:
      self.o = o
      
  def imprime(self):
    print("Posicion: ({},{}) orientacion: ({})".format(self.x,self.y,self.o))
     
  def getPositionX(self):
    return self.x
  
  def getPositionY(self):
    return self.y
  
  def getOrientation(self):
    return self.o
  
  def setPositionX(self,x):
    self.x = x
  
  def setPositionY(self,y):
    self.y = y
  
  def setOrientation(self,o):
    self.o = o
    
  def __repr__(self):
    return 'Posicion: ({},{}) orientacion: ({})'.format(self.x,self.y,self.o)
                                                      
  def __eq__(self, other):
    return self.x == other.x and self.y == other.y and self.o == other.o
  
  def __hash__(self):
    return hash(self.x,self.y,self.o)

  
  

In [5]:
class Laberinto(Problem):
  
  #dirmov = set(['adelante','atras','izquierda','derecha'])
  giros = set(['girizquierda','girderecha'])
  #actions = dirmov.union(giros)
  ori = ('N','E','S','W')
  
  def __init__(self, initial, goal,maze):
    Problem.__init__(self,initial,goal)
    self.maze = maze
    
  def checkcells(self,state):
    """ Comprobar celdas libres
        Regresa el conjunto de acciones posibles si el agente tuviera orientacion norte
        #Funcion antes llamada getactions#
    """
    freecells = set()
    x = state.getPositionX()
    y = state.getPositionY()
    
    if self.maze[x-1][y] == 0:
      freecells.add('adelante')
    if self.maze[x+1][y] == 0:
      freecells.add('atras')
    if self.maze[x][y-1] == 0:
      freecells.add('izquierda')
    if self.maze[x][y+1] == 0:
      freecells.add('derecha')
    
    return freecells
  
  def isIntersection(self, freecells):
    """ Comprobar comprueba si la celda actual es una interseccion
        Regresa bool
    """
    n = len(freecells)

    if n == 2:
      iscorridor = freecells.isdisjoint({'adelante','atras'}) or freecells.isdisjoint({'izquierda','derecha'})
      return not iscorridor
    else:
      return n > 2
    
  def actions(self, state):
    
    freecells = self.checkcells(state)

    #si la celda actual es una interseccion el agente solo puede girar
    if self.isIntersection(freecells):
      return list(self.giros)
    else:
      return list(freecells)
    
  def giroDer(self,o):
    i = self.ori.index(o)
    if i<3:
      return self.ori[i+1]
    else:
      return self.ori[0]
  
  def giroIzq(self,o):
    i = self.ori.index(o)
    if i>0:
      return self.ori[i-1]
    else:
      return self.ori[3]
    
  
  def result(self,state,action):
    x = state.getPositionX()
    y = state.getPositionY()
    o = state.getOrientation()
    
    temp = State(x,y,o)
    
    if action in self.giros:
      if action == 'girderecha':
        temp.setOrientation(self.giroDer(o))
      else:  
        temp.setOrientation(self.giroIzq(o))
    else:
      if (action == 'derecha'):
        temp.setPositionY(y+1)

      elif (action == 'izquierda'):
        temp.setPositionY(y-1)

      elif (action == 'adelante'):
        temp.setPositionX(x+1)

      else:
        temp.setPositionX(x-1)
      
    return temp
      
  def goal_test(self, state):
    
        if isinstance(self.goal, list):
            return state in self.goal  
        else:
            return state == self.goal  
          
  def path_cost(self, cost, state1, action, state2):
    return cost+1

In [6]:
lab = toymaze()

state0 = State(5,5,'N')
statef = State(5,0,'N')
esquina = State(1,9,'N')
pasillo = State(2,9,'N')
goal = [State(5,0,'N'),State(5,0,'S'),State(5,0,'E'),State(5,0,'W')]
tempgoal = [State(6,7,'N'),State(6,7,'S'),State(6,7,'E'),State(6,7,'W')]

maze = Laberinto(state0, goal, lab)

test = Laberinto(state0,goal,lab)
A = test.checkcells(state0)
B = test.checkcells(pasillo)
C = test.checkcells(esquina)

print('Prueba goal_test: {}'.format(test.goal_test(statef)))
print('Prueba isIntersection state0: {}'.format(test.isIntersection(A)))
print('Prueba actions state0: {}'.format(test.actions(state0)))
print('Prueba isIntersection pasillo: {}'.format(test.isIntersection(B)))
print('Prueba actions pasillo: {}'.format(test.actions(pasillo)))
print('Prueba isIntersection esquina: {}'.format(test.isIntersection(C)))
print('Prueba actions esquina: {}'.format(test.actions(esquina)))

print('\nResultado de las acciones de pasillo')
for i in test.actions(pasillo):
  print("accion: {}".format(i))
  stado = test.result(pasillo,i)
  stado.imprime()
  
print('\nResultado de las acciones de esquina')
nodoesq = Node(esquina)
for i in test.actions(esquina):
  print("accion: {}".format(i))
  stado.imprime()



++++++++++++++++++++++
++++++++++++++++++++++
++                  ++
++                  ++
++  ++  ++++++  ++  ++
++  ++  ++++++  ++  ++
++  ++  ++  ++  ++  ++
++  ++  ++  ++  ++  ++
++  ++++++  ++++++  ++
++  ++++++  ++++++  ++
    ++          ++  ++
    ++          ++  ++
++  ++++++++++  ++  ++
++  ++++++++++  ++  ++
++      ++          ++
++      ++          ++
++  ++++++  ++++++  ++
++  ++++++  ++++++  ++
++      ++      ++  ++
++      ++      ++  ++
++++++++++++++++++++++
++++++++++++++++++++++


Prueba goal_test: True
Prueba isIntersection state0: True
Prueba actions state0: ['girderecha', 'girizquierda']
Prueba isIntersection pasillo: False
Prueba actions pasillo: ['adelante', 'atras']
Prueba isIntersection esquina: True
Prueba actions esquina: ['girderecha', 'girizquierda']

Resultado de las acciones de pasillo
accion: adelante
Posicion: (3,9) orientacion: (N)
accion: atras
Posicion: (1,9) orientacion: (N)

Resultado de las acciones de esquina
accion: girderecha
Posicion: (1

In [7]:
pruebaNode=Node(state0)

In [8]:
repr(pruebaNode)

exp1 = pruebaNode.expand(maze)

for i in exp1:
  print(i)
  print("SOLUTION")
  print(i.solution())
  print("PATH")
  print(i.path())
  print("\n")
  

<Node Posicion: (5,5) orientacion: (E), parent_action: girderecha, depth 1>
SOLUTION
['girderecha']
PATH
[<Node Posicion: (5,5) orientacion: (N), parent_action: None, depth 0>, <Node Posicion: (5,5) orientacion: (E), parent_action: girderecha, depth 1>]


<Node Posicion: (5,5) orientacion: (W), parent_action: girizquierda, depth 1>
SOLUTION
['girizquierda']
PATH
[<Node Posicion: (5,5) orientacion: (N), parent_action: None, depth 0>, <Node Posicion: (5,5) orientacion: (W), parent_action: girizquierda, depth 1>]


